In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [3]:
summoner_name = "쪼렙이다말로하자"

match = pd.read_json("data/data_summoner_match_data_{name}.json".format(name=summoner_name))
timeline = pd.read_json("data/data_summoner_match_timeline_data_{name}.json".format(name=summoner_name))
with open("data/data_summoner_{}.json".format(summoner_name), 'r') as f:
    summoner = json.load(f)

# 경기 관련 데이터

In [4]:
match_length = len(match["info"])
match_length

793

In [5]:
info = match["info"]

participants = []
teams = []
for i in range(match_length):
    participants.append(info[i]["participants"])
    teams.append(info[i]["teams"])
    del(info[i]["participants"])
    del(info[i]["teams"])

In [6]:
df_info = pd.json_normalize(info)
df_info_1 = df_info[["gameDuration", "gameMode", "gameType", "gameVersion", "mapId"]].copy()    # 원본데이터는 유지
display(df_info_1.head(10))

,gameDuration,gameMode,gameType,gameVersion,mapId
0,1865764,CLASSIC,MATCHED_GAME,11.19.398.9466,11
1,1261439,CLASSIC,MATCHED_GAME,11.19.398.9466,11
2,1583363,CLASSIC,MATCHED_GAME,11.19.398.9466,11
3,2304330,CLASSIC,MATCHED_GAME,11.19.398.9466,11
4,939701,CLASSIC,MATCHED_GAME,11.19.398.9466,11
5,1471143,CLASSIC,MATCHED_GAME,11.19.398.9466,11
6,1945596,CLASSIC,MATCHED_GAME,11.19.398.9466,11
7,1038762,CLASSIC,MATCHED_GAME,11.19.398.9466,11
8,1371736,CLASSIC,MATCHED_GAME,11.19.398.9466,11
9,1542991,CLASSIC,MATCHED_GAME,11.19.398.9466,11


In [7]:
df_participants = pd.json_normalize(participants)
display(df_participants.head(1))

,0,1,2,3,4,5,6,7,8,9
0,"{'assists': 5, 'baronKills': 0, 'bountyLevel':...","{'assists': 10, 'baronKills': 0, 'bountyLevel'...","{'assists': 7, 'baronKills': 0, 'bountyLevel':...","{'assists': 8, 'baronKills': 0, 'bountyLevel':...","{'assists': 10, 'baronKills': 0, 'bountyLevel'...","{'assists': 13, 'baronKills': 0, 'bountyLevel'...","{'assists': 14, 'baronKills': 2, 'bountyLevel'...","{'assists': 7, 'baronKills': 0, 'bountyLevel':...","{'assists': 7, 'baronKills': 0, 'bountyLevel':...","{'assists': 20, 'baronKills': 0, 'bountyLevel'..."


In [8]:
# 표본의 게임 속 index 구하기
summoner_index = df_participants.apply(lambda row: np.where(row.map(lambda col: col["puuid"] == summoner["puuid"]))[0][0], axis=1)
# 표본의 게임 속 teamId 구하기
summoner_team = ["100" if summoner_index[x]<5 else "200" for x in range(match_length)]
# info_1 데이터프레임에 저장
df_info_1["index"] = summoner_index
df_info_1["team"] = summoner_team
display(df_info_1.head(10))

,gameDuration,gameMode,gameType,gameVersion,mapId,index,team
0,1865764,CLASSIC,MATCHED_GAME,11.19.398.9466,11,9,200
1,1261439,CLASSIC,MATCHED_GAME,11.19.398.9466,11,9,200
2,1583363,CLASSIC,MATCHED_GAME,11.19.398.9466,11,4,100
3,2304330,CLASSIC,MATCHED_GAME,11.19.398.9466,11,2,100
4,939701,CLASSIC,MATCHED_GAME,11.19.398.9466,11,7,200
5,1471143,CLASSIC,MATCHED_GAME,11.19.398.9466,11,2,100
6,1945596,CLASSIC,MATCHED_GAME,11.19.398.9466,11,4,100
7,1038762,CLASSIC,MATCHED_GAME,11.19.398.9466,11,7,200
8,1371736,CLASSIC,MATCHED_GAME,11.19.398.9466,11,2,100
9,1542991,CLASSIC,MATCHED_GAME,11.19.398.9466,11,9,200


In [9]:
df_teams = pd.json_normalize(teams)
df_teams.columns = ["100", "200"]   # teamId
display(df_teams.head(5))

,100,200
0,"{'bans': [{'championId': 412, 'pickTurn': 1}, ...","{'bans': [{'championId': 412, 'pickTurn': 6}, ..."
1,"{'bans': [{'championId': 76, 'pickTurn': 1}, {...","{'bans': [{'championId': 55, 'pickTurn': 6}, {..."
2,"{'bans': [{'championId': 114, 'pickTurn': 1}, ...","{'bans': [{'championId': 104, 'pickTurn': 6}, ..."
3,"{'bans': [{'championId': -1, 'pickTurn': 1}, {...","{'bans': [{'championId': 7, 'pickTurn': 6}, {'..."
4,"{'bans': [{'championId': 91, 'pickTurn': 1}, {...","{'bans': [{'championId': -1, 'pickTurn': 6}, {..."


In [10]:
# 이긴 팀의 teamId 구하기
teams_win = df_teams.apply(lambda row: "100" if row.map(lambda col: col["win"])[0] else "200", axis=1)
df_teams_1 = df_teams.copy()
df_teams_1["win"] = teams_win

## 데이터 저장

In [11]:
df_info_1.to_csv("data/{}/info.csv".format(summoner_name), index=False)
df_participants.to_json("data/{}/participants.json".format(summoner_name))
df_teams_1.to_json("data/{}/teams.json".format(summoner_name))

# 인베이드 관련 데이터

In [12]:
match_invade = 0

## 이벤트 분석

In [13]:
12200 // 4000

3

In [25]:
is_invade = [False]*match_length
invade_timestamp = [0]*match_length           
invade_position = np.array([0 for _ in range(64)]).reshape(8,8)
invade_kill_blue = [0]*match_length           # invade시 발생한 블루팀의 킬 수
invade_kill_red = [0]*match_length            # invade시 발생한 레드팀의 킬 수
invade_win_team = [0]*match_length            # invade가 발생한 게임에서 이긴 팀의 teamId
total_gold_blue_3_min = [0]*match_length
total_gold_red_3_min = [0]*match_length
gold_gap = [0]*match_length
killer = [0]*match_length

for match_index in range(match_length):
    match_time_length = len(timeline.iloc[match_index].info['frames'])
    
    events = [timeline.iloc[match_index].info["frames"][t]["events"] for t in range(match_time_length)]
    participants_info = [timeline.iloc[match_index].info["frames"][t]["participantFrames"] for t in range(match_time_length)]

    # 1~2분대의 kill event
    if len(events) < 3: 
        print("다시하기 발생!")
        continue    # 게임이 3분 이내로 끝났을 경우를 제외 
    for t in range(1,3):  # 1 ~ 2 분 사이
        killers = []
        for e in events[t]:    
            if e["type"] == "CHAMPION_KILL":
                # print(e)
                if is_invade[match_index] == False: 
                    is_invade[match_index] = True
                    match_invade += 1
                if invade_timestamp[match_index] == 0: invade_timestamp[match_index] = e['timestamp']
                if e['killerId'] <= 5:
                    invade_kill_blue[match_index] += 1
                elif e['killerId'] > 5:
                    invade_kill_red[match_index] += 1
                p_x, p_y = e['position']['x'], e['position']['y']
                invade_position[(16000-p_y)//2000][p_x//2000] += 1
                killers.append(e['killerId'])
    if is_invade[match_index]:
        killer[match_index] = killers
        for i in range(1,6):
            total_gold_blue_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        for i in range(6,11):
            total_gold_red_3_min[match_index] += participants_info[3][str(i)]['totalGold']
        gold_gap[match_index] = total_gold_blue_3_min[match_index] - total_gold_red_3_min[match_index]
        
        # use df_teams
        invade_win_team[match_index] = 100 if df_teams["100"][match_index]["win"] else 200
        
        print("--------------------------------------------------")
        print("invade detected: %d" % match_index)
        print("timestamp(minutes): %.2f" % (invade_timestamp[match_index]/60000))
        print("< kills >\n  blue : %d , red : %d"% (invade_kill_blue[match_index], invade_kill_red[match_index]))
        print("< total_gold_3_min >\n  blue: %d , red: %d"% (total_gold_blue_3_min[match_index], total_gold_red_3_min[match_index]))
        print("< gold_gap >\n  %d"% gold_gap[match_index])
        print("< win >\n  %s"% ("blue" if invade_win_team[match_index] == 100 else "red"))

--------------------------------------------------
invade detected: 2
timestamp(minutes): 1.89
< kills >
  blue : 0 , red : 1
< total_gold_3_min >
  blue: 4216 , red: 5094
< gold_gap >
  -878
< win >
  red
--------------------------------------------------
invade detected: 11
timestamp(minutes): 1.87
< kills >
  blue : 0 , red : 1
< total_gold_3_min >
  blue: 4419 , red: 5324
< gold_gap >
  -905
< win >
  blue
--------------------------------------------------
invade detected: 16
timestamp(minutes): 1.59
< kills >
  blue : 3 , red : 0
< total_gold_3_min >
  blue: 5579 , red: 4008
< gold_gap >
  1571
< win >
  blue
--------------------------------------------------
invade detected: 17
timestamp(minutes): 1.75
< kills >
  blue : 2 , red : 1
< total_gold_3_min >
  blue: 5063 , red: 4624
< gold_gap >
  439
< win >
  blue
--------------------------------------------------
invade detected: 21
timestamp(minutes): 1.56
< kills >
  blue : 1 , red : 0
< total_gold_3_min >
  blue: 4798 , red: 428

In [16]:
df_invade_position = pd.DataFrame(invade_position)
df_invade_position

,0,1,2,3,4,5,6,7
0,0,0,1,0,0,0,0,0
1,4,10,6,2,0,0,0,0
2,7,6,2,4,2,0,0,0
3,0,7,9,10,7,4,1,0
4,0,0,6,24,8,13,6,0
5,0,0,7,22,9,19,10,2
6,0,0,0,10,15,13,30,0
7,0,0,0,4,3,21,10,0


In [26]:
df_invade = pd.DataFrame([
    is_invade,
    invade_timestamp,
    invade_kill_blue,
    invade_kill_red,
    invade_win_team,
    total_gold_blue_3_min,
    total_gold_red_3_min,
    gold_gap,
    killer,
    df_info_1["gameMode"]]).transpose()

df_invade.columns = [
    "is_invade",
    "invade_timestamp",
    "invade_kill_blue",
    "invade_kill_red",
    "invade_win_team",
    "total_gold_blue_3_min",
    "total_gold_red_3_min",
    "gold_gap",
    "killer_id",
    "gameMode"]

display(df_invade[(df_invade.is_invade==True) & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])

# 인베가 일어났고, 블루가 레드보다 킬을 많이 딴 경기에서 블루가 이길 확률 
print("%.2f%%"%(
    len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red) 
        & (df_invade.invade_win_team == 100)]) 
    / len(df_invade[
        (df_invade.is_invade==True) 
        & (df_invade.invade_kill_blue > df_invade.invade_kill_red)])
    * 100)
)


,is_invade,invade_timestamp,invade_kill_blue,invade_kill_red,invade_win_team,total_gold_blue_3_min,total_gold_red_3_min,gold_gap,killer_id,gameMode
16,True,95344,3,0,100,5579,4008,1571,"[3, 3, 2]",CLASSIC
17,True,105263,2,1,100,5063,4624,439,"[10, 1, 1]",CLASSIC
21,True,93590,1,0,100,4798,4289,509,[1],CLASSIC
25,True,66227,2,0,100,5910,4207,1703,"[4, 2]",CLASSIC
35,True,103481,1,0,100,4908,4408,500,[2],CLASSIC
...,...,...,...,...,...,...,...,...,...,...
763,True,53253,1,0,100,5072,4598,474,[],CLASSIC
780,True,60967,3,0,100,6290,4139,2151,"[4, 5, 3]",CLASSIC
784,True,68454,1,0,200,4843,4312,531,[1],CLASSIC
788,True,57358,2,0,100,5395,5209,186,[2],CLASSIC


71.43%


In [ ]:
match_invade

184

In [27]:
df_invade.to_json("data/{}/invade.json".format(summoner_name))
df_invade_position.to_csv("data/{}/invade_position.csv".format(summoner_name), index=False)